In [1]:
import pandas as pd
from sqlalchemy.types import VARCHAR, INTEGER, FLOAT
from sqlalchemy import text
from sqlalchemy import create_engine


In [2]:
# connect to the database
engine = create_engine('mssql+pyodbc://WagnerProdAGL1/Targit_DM?driver=ODBC+Driver+17+for+SQL+Server', echo=False, fast_executemany=True)

In [4]:
# Read OTIF excel
df_OTIF_Stage = pd.read_excel('..\Files\CCPA Order Detail Wagner June 2024.xlsx', sheet_name='CCPA Order Detail Wagner June 2')

In [5]:
df_OTIF_Stage

,Rpt Dt,Frcst Enty Dlr Cd,Main Dlr Cd,Dlr Cd Emer,Inv Str No,Str No,Sls Chan,End Cust No,End Cust Nm,End Cust Ord No,...,Cof Svc Elig Ind,Cof Non Elig Rsn,Otif Svc Elig Ind,Otif Non Elig Rsn,Otif Ind,Teps Ind,Ces Ind,Key Cat Account Num,Parent Customer No,Cost Center
0,2024-06-05,E250,E250,E250,0,0,WO,0ICS628,DYNO/TRANS TEST SERVICE,AGS184898,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,CS
1,2024-06-27,E250,E250,E250,0,0,PS,2709,AMES CONSTRUCTION INC,AGC693033,...,Y,NaN,Y,NaN,Y,NaN,NaN,H64M,NaN,DN
2,2024-06-12,E250,E250,E260,9A,9A,OTC,99940,WAGNER CASH/WINDSOR,9AC004924,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,PD
3,2024-06-19,E250,E250,E260,9A,9A,PS,37514,GOODELL MACHINERY & CONST,9AC005773,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,PD
4,2024-06-27,E250,E250,E253,3,3,OTC,78357,SCHULZ FARMS LLC,03C534807,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,3P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36568,2024-06-25,E250,E258,E25J,60,60,WO,60WR625,SHOP SUPPLIES,60S008615,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,WR
36569,2024-06-27,E250,E258,E258,59,59,WO,0IWR500,WAGNER RENTS CAT REPAIRS,59S022120,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,WR
36570,2024-06-22,E250,E258,E25K,57,57,OTC,37618,GOULD CONSTRUCTION INC,57C057189,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,PD
36571,2024-06-10,E250,E258,E25J,60,60,OTC,49895,JSMX INC,60C059785,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,47849.0,PD


### Logic to getting the calculations made

In [6]:
# replace sapces with underscores in the columns
df_OTIF_Stage.columns = df_OTIF_Stage.columns.str.replace(' ','_')
df_OTIF_Stage['Inv_Str_No'] = df_OTIF_Stage['Inv_Str_No'].astype(str).str.zfill(2)
df_OTIF_Stage['Str_No'] = df_OTIF_Stage['Str_No'].astype(str).str.zfill(2)

# # Calculate count of 'Y' in column 'OTIL LINES' for each dealer code
# df_count_ones = df_OTIF_Stage[df_OTIF_Stage['Otif_Ind'] == 'Y'].groupby('Dlr_Cd_Emer')['Otif_Ind'].count().reset_index()
# df_count_ones = df_count_ones.rename(columns={'Otif_Ind': 'Y'})

# # Calculate total count in column 'OTIL LINES' for each dealer code
# df_total_count = df_OTIF_Stage.groupby('Dlr_Cd_Emer')['Otif_Ind'].count().reset_index()
# df_total_count = df_total_count.rename(columns={'Otif_Ind': 'Grand_Total'})

# # # Merge the two dataframes on 'Dlr_Cd_Emer'
# df_result = pd.merge(df_OTIF_Stage, df_count_ones, on='Dlr_Cd_Emer', how='left').fillna(0)
# df_result = pd.merge(df_result, df_total_count, on='Dlr_Cd_Emer', how='left').fillna(0)

# # Fill NaN values with 0 in case there are no '1' occurrences for a dealer code
# df_result = df_result.fillna(0)

# # Calculate the Percentage column
# df_result['Percentage'] = (df_result['Y'] / df_result['Grand_Total']) * 100

df_result = df_OTIF_Stage

# Convert 'Dlr_Cd_Emer' to string explicitly
df_result['Dlr_Cd_Emer'] = df_result['Dlr_Cd_Emer'].astype(str)

In [7]:
df_result

,Rpt_Dt,Frcst_Enty_Dlr_Cd,Main_Dlr_Cd,Dlr_Cd_Emer,Inv_Str_No,Str_No,Sls_Chan,End_Cust_No,End_Cust_Nm,End_Cust_Ord_No,...,Cof_Svc_Elig_Ind,Cof_Non_Elig_Rsn,Otif_Svc_Elig_Ind,Otif_Non_Elig_Rsn,Otif_Ind,Teps_Ind,Ces_Ind,Key_Cat_Account_Num,Parent_Customer_No,Cost_Center
0,2024-06-05,E250,E250,E250,00,00,WO,0ICS628,DYNO/TRANS TEST SERVICE,AGS184898,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,CS
1,2024-06-27,E250,E250,E250,00,00,PS,2709,AMES CONSTRUCTION INC,AGC693033,...,Y,NaN,Y,NaN,Y,NaN,NaN,H64M,NaN,DN
2,2024-06-12,E250,E250,E260,9A,9A,OTC,99940,WAGNER CASH/WINDSOR,9AC004924,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,PD
3,2024-06-19,E250,E250,E260,9A,9A,PS,37514,GOODELL MACHINERY & CONST,9AC005773,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,PD
4,2024-06-27,E250,E250,E253,03,03,OTC,78357,SCHULZ FARMS LLC,03C534807,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,3P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36568,2024-06-25,E250,E258,E25J,60,60,WO,60WR625,SHOP SUPPLIES,60S008615,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,WR
36569,2024-06-27,E250,E258,E258,59,59,WO,0IWR500,WAGNER RENTS CAT REPAIRS,59S022120,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,WR
36570,2024-06-22,E250,E258,E25K,57,57,OTC,37618,GOULD CONSTRUCTION INC,57C057189,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,PD
36571,2024-06-10,E250,E258,E25J,60,60,OTC,49895,JSMX INC,60C059785,...,Y,NaN,Y,NaN,Y,NaN,NaN,NaN,47849.0,PD


In [28]:
# # Calculate count of 'Y' in column 'OTIL LINES' for each dealer code
# df_count_ones = df_OTIF_Stage[df_OTIF_Stage['Otif Ind'] == 'Y'].groupby('Dlr Cd Emer')['Otif Ind'].count().reset_index()
# df_count_ones = df_count_ones.rename(columns={'Otif Ind': 'Y'})

# # Calculate total count in column 'OTIL LINES' for each dealer code
# df_total_count = df_OTIF_Stage.groupby('Dlr Cd Emer')['Otif Ind'].count().reset_index()
# df_total_count = df_total_count.rename(columns={'Otif Ind': 'Grand Total'})

# # Merge the two dataframes on 'Dlr Cd Emer'
# df_result = pd.merge(df_count_ones, df_total_count, on='Dlr Cd Emer', how='right')

# # Fill NaN values with 0 in case there are no '1' occurrences for a dealer code
# df_result = df_result.fillna(0)

# # Calculate the Percentage column
# df_result['Percentage'] = (df_result['Y'] / df_result['Grand Total']) * 100

# # Convert 'Dlr Cd Emer' to string explicitly
# df_result['Dlr Cd Emer'] = df_result['Dlr Cd Emer'].astype(str)
# df_result

In [30]:
# Specify data types for each column
dtype_mapping = {
    'Dlr Cd Emer': VARCHAR,
    'Y': INTEGER,
    'Grand Total': INTEGER,
    'Percentage': FLOAT,
}


In [8]:
# Convert the DataFrame to a table in the dbo schema without explicit data types
df_result.to_sql('OTIF', con=engine, index=False, if_exists='replace', schema='dbo', method = 'multi', chunksize=60)

36573

In [ ]:
# # Convert the DataFrame to a table in the dbo schema without explicit data types
# df_result.to_sql('OTIF_Stage', con=engine, index=False, if_exists='replace', schema='dbo')

In [32]:
# # Define the SQL query to create the final table
# sql_create_table_query = """
# CREATE TABLE [Targit_DM].[dbo].[OTIF_data] (
#     [Dealer_Code] VARCHAR(25),
#     [Rollup_Store] VARCHAR(10),
#     [Y] INT,
#     [Grand_Total] INT,
#     [Percentage] FLOAT
# )
# """

In [33]:
# # Define the SQL query to insert into the final table
# sql_insert_query = """
# INSERT INTO [Targit_DM].[dbo].[OTIF_data]
# SELECT 
#        t1.[Dlr Cd Emer]
#       ,t2.[Rollup_Store]
#       ,t1.[Y]
#       ,t1.[Grand Total]
#       ,t1.[Percentage]
# FROM [dbo].[OTIF_Stage] t1
# LEFT JOIN [Daisy\Wagner].[XREF].[dbo].[Parts_KPIDealerCodeStoreRollup] t2
# ON t1.[Dlr Cd Emer] = t2.Dealer_Code
# """

In [34]:
# # Define the SQL query to truncate the staging table
# sql_truncate_staging = "TRUNCATE TABLE [Targit_DM].[dbo].[OTIF_Stage]"

In [35]:
# # Execute the SQL queries
# with engine.connect() as connection:
#     # Create the final table
#     connection.execute(text(sql_create_table_query))
    
#     # Insert into final table
#     connection.execute(text(sql_insert_query))
    
#     # Truncate staging table
#     connection.execute(text(sql_truncate_staging))

#     # Commit the transaction
#     connection.commit()


In [ ]:
# Write df_OTIL to SQL Server (replace 'final_table' with your actual table name)
# df_OTIL.to_sql('Parts_OTIL', con=engine, index=False, if_exists='replace')


# Write df_OTIL to Excel
df_OTIL.to_excel('OTIL_Testing.xlsx', index=False)